In [3]:
#import modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#import da
df = pd.read_csv('Trabalho.csv')
df.head(10)

,Unnamed: 0,Y,X1,X2,X3,X4
0,1,1.448136,0.291702,2.587011,2.564149,2.233988
1,2,3.001715,0.517189,3.462381,3.414562,-1.518295
2,3,3.168068,0.607613,4.052388,3.974191,1.875687
3,4,2.522207,1.615740,1.999990,1.880660,1.234311
4,5,0.085236,-0.992394,2.667726,2.611272,2.660299
5,6,3.293039,1.932170,3.196037,3.166793,0.232232
6,7,1.033810,0.044219,1.461748,1.343791,1.160812
7,8,2.695423,1.913302,2.205511,2.310261,-0.912108
8,9,3.626690,2.768036,2.038721,2.114006,-1.871051
9,10,2.611396,1.399541,1.239386,1.267726,0.136095
